In [6]:
import cortex
pts, polys = cortex.db.get_surf('subj01', 'wm', merge=True)
cortex.database.default_filestore

import torch
import sys
import numpy as np
from copy import deepcopy

sys.path.append('/user_data/mmhender/imStat/code/')
from utils import nsd_utils, plot_utils
from utils.plot_utils import get_full_surface

In [5]:
# fit_fn = '/user_data/mmhender/imStat/model_fits/S01_surface/texture_pyramid_ridge_4ori_4sf/Aug-31-2021_1835_32/all_fit_params'
fit_fn = '/user_data/mmhender/imStat/model_fits/S01_surface/bdcn_pca_fused/Sep-08-2021_0032_26/all_fit_params'
out = torch.load(fit_fn)

voxel_mask = out['voxel_mask']
brain_nii_shape = out['brain_nii_shape']
image_order = out['image_order']
voxel_idx = out['voxel_index'][0]
voxel_ncsnr = out['voxel_ncsnr'].ravel()[voxel_idx]
noise_ceiling = nsd_utils.ncsnr_to_nc(voxel_ncsnr)

best_lambdas = out['best_lambdas']
best_losses = out['best_losses']
lambdas = out['lambdas']

val_cc_partial = out['val_cc']
# Note i'm NOT using the thing that actually is in the field val_r2, 
# bc that is coefficient of determination which gives poor results for ridge regression.
# instead using the signed squared correlation coefficient for r2/var explained.
val_r2_partial = np.sign(val_cc_partial)*val_cc_partial**2

# feature_info = deepcopy(out['feature_info'])
# feature_type_labels, feature_type_names = feature_info
# n_feature_types = len(feature_type_names)

# Compute variance explained by each feature type - how well does the model without that feature type
# do, compared to the model with all features? 
# var_expl = np.tile(np.expand_dims(val_r2_partial[:,0], axis=1), [1,n_feature_types]) - val_r2_partial[:,1:] 
# feature_adj_labels = np.arange(0, n_feature_types,1)
 
best_params = out['best_params']
best_models, weights, bias, features_mt, features_st, best_model_inds = best_params

best_models_partial = best_models
best_models = best_models[:,0,:] # using only the "full" model for now
val_cc = val_cc_partial[:,0]
val_r2 = val_r2_partial[:,0]
best_lambdas = best_lambdas[:,0]
best_losses = best_losses[:,0]

best_ecc  = np.sqrt(np.square(best_models[:,0]) + np.square(best_models[:,1]))
best_ang  = np.arctan2(best_models[:,1], best_models[:,0])
best_size = best_models[:,2]

n_voxels = np.shape(weights)[0]
n_features_total = np.shape(weights)[1]
n_partial_models = np.shape(weights)[2]

In [2]:
from utils.plot_utils import get_full_surface


ModuleNotFoundError: No module named 'utils'

In [8]:
substr = 'subj01'

cmin = 0.0
cmax = 0.8
rmin = 0.0
rmax = 0.6
vemin = -0.05
vemax = 0.05

eccen_deg = 8.4

dat2plot = {'pRF eccentricity': cortex.Vertex(data = get_full_surface(best_ecc*eccen_deg, voxel_mask), subject=substr, cmap='PRGn', vmin=0, vmax=7), \
            'pRF angle': cortex.Vertex(data = get_full_surface(best_ang*180/np.pi, voxel_mask), subject=substr, cmap='Retinotopy_RYBCR', vmin=-180, vmax=180), \
            'pRF size': cortex.Vertex(data = get_full_surface(best_size*eccen_deg, voxel_mask), subject=substr, cmap='PRGn', vmin=0, vmax=4), \
            'Correlation (validation set)': cortex.Vertex(data = get_full_surface(val_cc, voxel_mask), subject=substr, cmap='PuBu', vmin=cmin, vmax=cmax), \
            'R2 (validation set)': cortex.Vertex(data = get_full_surface(val_r2, voxel_mask), subject=substr, cmap='PuBu', vmin=rmin, vmax=rmax), \
           }

# for fi, ff in enumerate(feature_info[1]):
#     dat2plot['Var expl: %s'%ff] = cortex.Vertex(data = get_full_surface(var_expl[:,fi], voxel_mask), subject=substr, vmin=vemin, vmax=vemax)
    
dat2plot.keys()

dict_keys(['pRF eccentricity', 'pRF angle', 'pRF size', 'Correlation (validation set)', 'R2 (validation set)'])

In [9]:
# Open the webviewer
# fitting_type = 'Steerable pyramid texture model'
fitting_type = 'BDCN edge map features'
cortex.webshow(dat2plot, open_browser=True, port=44445, description = '%s NSD, %s'%(substr, fitting_type))

Started server on port 44445


<JS: window.viewer>

Stopping server


In [6]:
get_full_surface(best_ecc*eccen_deg, voxel_mask)

array([1.23986685, 1.23986685, 1.41784199, ...,        nan,        nan,
              nan])